In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-02-21 19:38:42--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2022-02-21 19:38:42 (5.99 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Load Amazon Data into Spark DataFrame

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Personal_Care_Appliances_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
pca_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Personal_Care_Appliances_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
pca_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32114233|R1QX6706ZWJ1P5|B00OYRW4UE|     223980852|Elite Sportz Exer...|Personal_Care_App...|          5|            0|          0|   N|                Y|Good quality. Shi...|Exactly as descri...| 2015-08-31|
|         US|   18125776|R3QWMLJHIW6P37|B0000537JQ|     819771537|     Ezy Dose Weekly|Personal_Care_App

# Size of Data

In [ ]:
pca_df.count()

85981

In [ ]:
pca_df.describe().show()

+-------+-----------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+------------------+------------------+------------------+-----+-----------------+--------------------+--------------------+-----------+
|summary|marketplace|         customer_id|     review_id|          product_id|      product_parent|       product_title|    product_category|       star_rating|     helpful_votes|       total_votes| vine|verified_purchase|     review_headline|         review_body|review_date|
+-------+-----------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+------------------+------------------+------------------+-----+-----------------+--------------------+--------------------+-----------+
|  count|      85981|               85981|         85981|               85981|               85981|               85981|               85981|             85981|         

In [ ]:
pca_df.summary().show()

+-------+-----------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+------------------+------------------+------------------+-----+-----------------+--------------------+--------------------+-----------+
|summary|marketplace|         customer_id|     review_id|          product_id|      product_parent|       product_title|    product_category|       star_rating|     helpful_votes|       total_votes| vine|verified_purchase|     review_headline|         review_body|review_date|
+-------+-----------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+------------------+------------------+------------------+-----+-----------------+--------------------+--------------------+-----------+
|  count|      85981|               85981|         85981|               85981|               85981|               85981|               85981|             85981|         

# Cleaned up DataFrames to match tables

In [ ]:
from pyspark.sql.functions import to_date
# Review DataFrame
review_id_df = pca_df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1QX6706ZWJ1P5|   32114233|B00OYRW4UE|     223980852| 2015-08-31|
|R3QWMLJHIW6P37|   18125776|B0000537JQ|     819771537| 2015-08-31|
|R14Z1VR1N0Z9G6|   19917519|B00HXXO332|     849307176| 2015-08-31|
| R25ZRJL0GH0U0|   18277171|B00EOB0JA2|     700864740| 2015-08-31|
|R3837KYH7AZNIY|    2593270|B00OC2O1UC|     794298839| 2015-08-31|
|R2MN0QYCY6EVIV|    2592955|B00HES9CMS|     318730927| 2015-08-31|
|R3AN2UJ1D42ZS0|   15168265|B0016BFR4G|     887476137| 2015-08-31|
|R3U29ZLUWEEK4M|   13761624|B00K504UUG|     458706868| 2015-08-31|
|R16ZDMJJHK796C|   37070734|B00HES9CMS|     318730927| 2015-08-31|
| RRRDOEJZD1Y22|   29615023|B00P6TUO5G|     170248843| 2015-08-31|
|R2KR5ZEAS859DK|   47893062|B0006VJ6TO|     412568457| 2015-08-31|
| RR7PGQY763IHF|    2582596|B00H9L7VIW|     851045898| 2015-08

In [ ]:
# drop the duplicates
products_df = pca_df.select(["product_id", "product_title"]).drop_duplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00J7G8EL0|GUM Expanding Flo...|
|B000BGI4DS|MedWeigh MS-2510 ...|
|B00CADD4R6|Cigarette Injecto...|
|B008LEPP3O|Cytomax Powder-Or...|
|B00NTRVVA4|Beauty Wig World ...|
|B000KE8ASK|Inc Magnetic Peda...|
|B00U11YXD8|SG Rubber Foam Mu...|
|B008Y7EZMU|"ABC Products" - ...|
|B00075VVYC|Reizen RL-350 Bra...|
|B005P2KHB0|AUTHENTIC - DESIG...|
|B0002UBEGW|SASSOON HAIR TRIM...|
|B00BHEO7RK|Recliner Risers S...|
|B00GIOTGPW|Hot Tools Dryer -...|
|B00GIX9GLW|Bundle - Doc John...|
|B00GOTNA3A|Wet Light Liquid ...|
|B003VJETUO|BooJee 6 Pk Booje...|
|B003BIFRFQ|Pjur Backdoor Rel...|
|B00HHP2QQ2|ULTRA DNA TM 50 S...|
|B001QGMXAW|Drive Medical Car...|
|B0021YY0YK|Medline ORT27200 ...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
# create review df
review_df = pca_df.select(["review_id","review_headline","review_body"])
review_df.show()


+--------------+--------------------+--------------------+
|     review_id|     review_headline|         review_body|
+--------------+--------------------+--------------------+
|R1QX6706ZWJ1P5|Good quality. Shi...|Exactly as descri...|
|R3QWMLJHIW6P37|          Five Stars|         It is great|
|R14Z1VR1N0Z9G6|It's really nice ...|It's really nice ...|
| R25ZRJL0GH0U0|           Two Stars|The kit works fin...|
|R3837KYH7AZNIY|          Four Stars|It works better t...|
|R2MN0QYCY6EVIV|      not bad at all|I added to my oat...|
|R3AN2UJ1D42ZS0|Mild, enjoyable t...|Husband drinks th...|
|R3U29ZLUWEEK4M|          Five Stars|       Good quality.|
|R16ZDMJJHK796C|        High Quality|This is high qual...|
| RRRDOEJZD1Y22|          we like it|Buying more produ...|
|R2KR5ZEAS859DK|          Five Stars|    Their best cane.|
| RR7PGQY763IHF|       Great product|Great product! Ma...|
| RN37YYZBP9BMP|Love the prouduct...|Love the prouduct...|
| R1UXGB70OR5TW|          Five Stars|          great buy

In [ ]:
# group customer id by customer count
#pca_df.groupby("customer_id").agg(count(review_id).as("review_count")).show()
#pca_df.groupby("customer_id").count().withColumnRenamed("count","review_count").sort("count").show()
customers_df = pca_df.groupby("customer_id").count().withColumnRenamed("count","customer_count").orderBy("customer_count",ascending=False)
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   50820654|            26|
|   28496704|            15|
|   43856165|            14|
|   37062164|            13|
|   48042126|            12|
|   46677169|            12|
|   33087196|            11|
|   22631069|            11|
|   16722770|            10|
|   44441036|             9|
|   52836416|             8|
|   53047863|             8|
|   52989182|             8|
|    3949232|             8|
|   51086982|             7|
|   37433724|             7|
|    6660431|             7|
|   36903892|             7|
|   53076177|             6|
|   52293338|             6|
+-----------+--------------+
only showing top 20 rows



In [ ]:
# select columns from vine_df
vine_df=pca_df.select(["review_id","star_rating","helpful_votes","total_votes","vine"])
vine_df.show()


+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1QX6706ZWJ1P5|          5|            0|          0|   N|
|R3QWMLJHIW6P37|          5|            0|          0|   N|
|R14Z1VR1N0Z9G6|          5|            1|          1|   N|
| R25ZRJL0GH0U0|          2|            0|          0|   N|
|R3837KYH7AZNIY|          4|            0|          1|   N|
|R2MN0QYCY6EVIV|          5|            0|          0|   N|
|R3AN2UJ1D42ZS0|          5|            0|          0|   N|
|R3U29ZLUWEEK4M|          5|            0|          0|   N|
|R16ZDMJJHK796C|          5|            0|          0|   N|
| RRRDOEJZD1Y22|          4|            0|          0|   N|
|R2KR5ZEAS859DK|          5|            0|          0|   N|
| RR7PGQY763IHF|          5|            0|          0|   N|
| RN37YYZBP9BMP|          5|            4|          4|   N|
| R1UXGB70OR5TW|          5|            

# Push to AWS RDS instance

In [ ]:
# set up properties to connect to AWS Postgres
mode = "append"
jdbc_url="jdbc:postgresql://bcscloudetl.c5odsdiuw6ip.us-east-1.rds.amazonaws.com:5432/EXAMPLE1"
config = {"user":"bcs_postgres", "password": "<password>", "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vines', mode=mode, properties=config)